## Adult Dataset

In [0]:
display(dbutils.fs.ls('/databricks-datasets/adult/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/adult/README.md,README.md,2672,1454697653000
dbfs:/databricks-datasets/adult/adult.data,adult.data,3974305,1444260537000
dbfs:/databricks-datasets/adult/adult.test,adult.test,2003132,1454373565000


In [0]:
df = (
  spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("/databricks-datasets/adult/adult.data")
)

In [0]:
display(df.head(5))

39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
37,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K


## Import Adult Dataset into Spark SQL Table

In [0]:
%sql

DROP TABLE IF EXISTS adult

In [0]:
%sql

CREATE TABLE adult (
  age DOUBLE,
  workclass STRING,
  fnlwgt DOUBLE,
  education STRING,
  education_num DOUBLE,
  marital_status STRING,
  occupation STRING,
  relationship STRING,
  race STRING,
  sex STRING,
  capital_gain DOUBLE,
  capital_loss DOUBLE,
  hours_per_week DOUBLE,
  native_country STRING,
  income STRING)
USING com.databricks.spark.csv
OPTIONS (path "/databricks-datasets/adult/adult.data", header "true")

In [0]:
spark.sql("SELECT * FROM adult LIMIT 5")

Out[6]: DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string]

## Data Schema for the adult dataset

In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField , IntegerType, StringType, DoubleType
 
adultSchema = StructType([
    StructField("age", DoubleType(), True),        
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", DoubleType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", DoubleType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),   
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", DoubleType(), True),   
    StructField("capital_loss", DoubleType(), True),
    StructField("hours_per_week", DoubleType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True),
 
])

## Alternative Approach Create DataFrame for the Adult dataset   

In [0]:
df = (
  spark.read
  .format("csv")
  .option("header", "false")
  .schema(adultSchema)
  .load("/databricks-datasets/adult/adult.data")
)

In [0]:
display(df.head(5))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [0]:
df.printSchema()

root
 |-- age: double (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)



In [0]:
display(df.sort(['age', 'education_num'], ascending=[True, True]).head(5))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
17.0,Private,270942.0,5th-6th,3.0,Never-married,Other-service,Other-relative,White,Male,0.0,0.0,48.0,Mexico,<=50K
17.0,Private,168807.0,7th-8th,4.0,Never-married,Craft-repair,Not-in-family,White,Male,0.0,0.0,45.0,United-States,<=50K
17.0,Private,168203.0,7th-8th,4.0,Never-married,Farming-fishing,Other-relative,Other,Male,0.0,0.0,40.0,Mexico,<=50K
17.0,Private,46402.0,7th-8th,4.0,Never-married,Sales,Own-child,White,Male,0.0,0.0,8.0,United-States,<=50K
17.0,?,127003.0,9th,5.0,Never-married,?,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

window_spec = Window.orderBy("age", "education_num")
df = df.withColumn("DenseRank", F.dense_rank().over(window_spec))
display(df[df['DenseRank'] <= 5].head(15))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,DenseRank
17.0,Private,270942.0,5th-6th,3.0,Never-married,Other-service,Other-relative,White,Male,0.0,0.0,48.0,Mexico,<=50K,1
17.0,Private,168807.0,7th-8th,4.0,Never-married,Craft-repair,Not-in-family,White,Male,0.0,0.0,45.0,United-States,<=50K,2
17.0,Private,168203.0,7th-8th,4.0,Never-married,Farming-fishing,Other-relative,Other,Male,0.0,0.0,40.0,Mexico,<=50K,2
17.0,Private,46402.0,7th-8th,4.0,Never-married,Sales,Own-child,White,Male,0.0,0.0,8.0,United-States,<=50K,2
17.0,Private,191260.0,9th,5.0,Never-married,Other-service,Own-child,White,Male,1055.0,0.0,24.0,United-States,<=50K,3
17.0,Private,211870.0,9th,5.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,6.0,United-States,<=50K,3
17.0,Local-gov,32124.0,9th,5.0,Never-married,Other-service,Own-child,Black,Male,0.0,0.0,9.0,United-States,<=50K,3
17.0,?,127003.0,9th,5.0,Never-married,?,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K,3
17.0,Private,221129.0,9th,5.0,Married-civ-spouse,Other-service,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K,3
17.0,?,275778.0,9th,5.0,Never-married,?,Own-child,White,Female,0.0,0.0,25.0,Mexico,<=50K,3


## Using SparkSQL

In [0]:
display(spark.sql("SELECT * FROM adult LIMIT 5"))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K


In [0]:
display(spark.sql("SELECT * FROM adult ORDER BY age asc, education_num asc LIMIT 5"))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
17.0,Private,270942.0,5th-6th,3.0,Never-married,Other-service,Other-relative,White,Male,0.0,0.0,48.0,Mexico,<=50K
17.0,Private,168807.0,7th-8th,4.0,Never-married,Craft-repair,Not-in-family,White,Male,0.0,0.0,45.0,United-States,<=50K
17.0,Private,168203.0,7th-8th,4.0,Never-married,Farming-fishing,Other-relative,Other,Male,0.0,0.0,40.0,Mexico,<=50K
17.0,Private,46402.0,7th-8th,4.0,Never-married,Sales,Own-child,White,Male,0.0,0.0,8.0,United-States,<=50K
17.0,?,127003.0,9th,5.0,Never-married,?,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K


In [0]:
display(spark.sql(
"""
SELECT
    *
FROM
    (
        SELECT
            *,
            DENSE_RANK() over (
                ORDER BY
                    age asc,
                    education_num asc
            ) as rank_
        FROM
            adult
    )
where
    rank_ <= 5
LIMIT 15 -- For displaying limited output
"""))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,rank_
17.0,Private,270942.0,5th-6th,3.0,Never-married,Other-service,Other-relative,White,Male,0.0,0.0,48.0,Mexico,<=50K,1
17.0,Private,168807.0,7th-8th,4.0,Never-married,Craft-repair,Not-in-family,White,Male,0.0,0.0,45.0,United-States,<=50K,2
17.0,Private,168203.0,7th-8th,4.0,Never-married,Farming-fishing,Other-relative,Other,Male,0.0,0.0,40.0,Mexico,<=50K,2
17.0,Private,46402.0,7th-8th,4.0,Never-married,Sales,Own-child,White,Male,0.0,0.0,8.0,United-States,<=50K,2
17.0,Private,191260.0,9th,5.0,Never-married,Other-service,Own-child,White,Male,1055.0,0.0,24.0,United-States,<=50K,3
17.0,Private,211870.0,9th,5.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,6.0,United-States,<=50K,3
17.0,Local-gov,32124.0,9th,5.0,Never-married,Other-service,Own-child,Black,Male,0.0,0.0,9.0,United-States,<=50K,3
17.0,?,127003.0,9th,5.0,Never-married,?,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K,3
17.0,Private,221129.0,9th,5.0,Married-civ-spouse,Other-service,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K,3
17.0,?,275778.0,9th,5.0,Never-married,?,Own-child,White,Female,0.0,0.0,25.0,Mexico,<=50K,3
